In [1]:
%load_ext zipline

In [2]:
%%zipline --start 2014-1-1 --end 2018-1-1 -o dma.pickle

import matplotlib.pyplot as plt
from zipline.api import order_target, record, symbol

def initialize(context):
    context.i = 0
    context.asset = symbol('000333')


def handle_data(context, data):
    # Skip first 300 days to get full windows
    context.i += 1
    if context.i < 300:
        return

    # Compute averages
    # data.history() has to be called with the same params
    # from above and returns a pandas dataframe.
    short_mavg = data.history(context.asset, 'price', bar_count=100, frequency="1d").mean()
    long_mavg = data.history(context.asset, 'price', bar_count=300, frequency="1d").mean()

    # Trading logic
    if short_mavg > long_mavg:
        # order_target orders as many shares as needed to
        # achieve the desired number of shares.
        order_target(context.asset, 100)
    elif short_mavg < long_mavg:
        order_target(context.asset, 0)

    # Save values for later inspection
    record(美的集团=data.current(context.asset, 'price'),
           short_mavg=short_mavg,
           long_mavg=long_mavg)


def analyze(context, perf):
    fig = plt.figure()
    ax1 = fig.add_subplot(211)
    perf.portfolio_value.plot(ax=ax1)
    ax1.set_ylabel('portfolio value in $')

    ax2 = fig.add_subplot(212)
    perf['美的集团'].plot(ax=ax2)
    perf[['short_mavg', 'long_mavg']].plot(ax=ax2)

    perf_trans = perf.ix[[t != [] for t in perf.transactions]]
    buys = perf_trans.ix[[t[0]['amount'] > 0 for t in perf_trans.transactions]]
    sells = perf_trans.ix[
        [t[0]['amount'] < 0 for t in perf_trans.transactions]]
    ax2.plot(buys.index, perf.short_mavg.ix[buys.index],
             '^', markersize=10, color='m')
    ax2.plot(sells.index, perf.short_mavg.ix[sells.index],
             'v', markersize=10, color='k')
    ax2.set_ylabel('price in $')
    plt.legend(loc=0)
    plt.show()


[2018-04-26 14:45:32.483952] INFO: Loader: Cache at /home/ldf/.zipline/data/000300_benchmark.csv does not have data from 1990-12-19 00:00:00+00:00 to 2018-04-25 00:00:00+00:00.

[2018-04-26 14:45:32.484944] INFO: Loader: Downloading benchmark data for '000300' from 1989-12-29 00:00:00+00:00 to 2018-04-25 00:00:00+00:00
[2018-04-26 14:45:32.586864] WARNING: Loader: Still don't have expected benchmark data for '000300' from 1989-12-29 00:00:00+00:00 to 2018-04-25 00:00:00+00:00 after redownload!
[2018-04-26 14:45:32.587937] INFO: Loader: Cache at /home/ldf/.zipline/data/treasury_curves_cn.csv does not have data from 2006-03-01 00:00:00+00:00 to 2018-04-25 00:00:00+00:00.

[2018-04-26 14:45:32.590081] INFO: Loader: Downloading treasury data for '000300' from 2006-03-01 00:00:00+00:00 to 2018-04-25 00:00:00+00:00


ValueError: setting an array element with a sequence.